### Edgar Moises Hernandez-Gonzalez
#### 20/04/20
#### Clasificacion de imagenes motoras en EEG con CNN-2D
#### Cargar modelo para evaluacion y predecir
##### Caracteristicas = Espectrogramas STFFT (3D)
##### Concatenacion vertical
##### Resize proporcional de 264x8

In [0]:
from google.colab import drive

In [0]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import cohen_kappa_score, confusion_matrix
from scipy.signal import spectrogram
import cv2
from keras.models import load_model

Using TensorFlow backend.


In [0]:
x_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RAW/MI-EEG-A01E.csv", header=None)
y_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RAW/EtiquetasA01E.csv", header=None)

In [0]:
print(x_test.shape)
print(y_test.shape)

(288, 22000)
(288, 1)


In [0]:
n_samples_test = len(y_test) #en caso de no tener el y_test se puede utilizar el x_test

print("n_samples_test :", n_samples_test)

n_samples_test : 288


In [0]:
n_classes = len(np.unique(y_test)) #esto se podria calcular asi n_clases = len(np.unique(y)), la y puede ser train o test

print("n_classes:", n_classes)

n_classes: 4


In [0]:
#codificar etiquetas con valores de 0 a nClases-1
y_test = LabelEncoder().fit_transform(y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
#unir 22 canales y resize de tamaño proporcional
def unir_espectrogramas_vertical_resize(data):
  dim = (8, 264) #ancho, alto
  fs = 250
  datos = np.zeros((288,264,8))
  temporal = np.zeros((528,4))
  for i in range(data.shape[0]): #288 muestras
    for j in range(22): #22 canales
      sig = data.iloc[i, j*1000:(j+1)*1000]
      f, t, Sxx = spectrogram(sig, fs) #espectrograma de STFFT
      temporal[j*24:(j+1)*24, :] = Sxx[8:32, :]
    resized = cv2.resize(temporal, dim, interpolation=cv2.INTER_AREA)
    datos[i] = resized
    if i % 100 == 0: #esto solo es para ver como avanza
      print(i)
  return datos

In [0]:
x_test = unir_espectrogramas_vertical_resize(x_test)

0
100
200


In [0]:
print(x_test.shape)

(288, 264, 8)


In [0]:
#convertir a float
x_test = x_test.astype('float32')

#escalar los valores en un rango de 0 a 1 (normalizar)
x_test /= 66.0

In [0]:
#convertir de 3D a 4D
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], x_test.shape[2], 1))

print(x_test.shape)

(288, 264, 8, 1)


In [0]:
model = load_model('/content/drive/My Drive/BCI-IV-2a/Modelos/Mejor_modelo_STFTDA01.hdf5')

In [0]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

In [0]:
print("loss: ", test_loss)
print("accuracy: ", test_acc)

loss:  0.8225199149714576
accuracy:  0.6736111044883728


In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 264, 8, 16)        160       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 132, 4, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 132, 4, 16)        2320      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 66, 2, 16)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2112)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               270464    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              